In [ ]:
import subprocess
import os
from datetime import datetime

mode = "clayg"
distances = [8]
start = 10**-4
end = 10**-2
step = 5 * 10**-5

cwd = os.getcwd()
base_dir = os.path.join(cwd, "data/treshold_plots")
os.makedirs(base_dir, exist_ok=True)

existing_ids = [
    int(f.split("-")[0]) for f in os.listdir(base_dir) 
    if os.path.isdir(os.path.join(base_dir, f)) and f.split("-")[0].isdigit()
]

next_id = max(existing_ids, default=0) + 1
# check if folder starting with that id already exists, if not create a new one with next available id
if not next_id in existing_ids:    
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = os.path.join(base_dir, f"{next_id}-{timestamp}")
    os.makedirs(output_dir, exist_ok=True)
else: 
    folder = [f for f in os.listdir(base_dir) if f.startswith(str(next_id))][0]
    output_dir = os.path.join(base_dir, folder)

print(output_dir)

for d in distances:
    output_file = os.path.join(output_dir, f"d_{d}.txt")
    print(output_file)
    exit(1)
    command = f"/home/tommaso-peduzzi/Documents/clayg/cmake-build-debug/clayg {d} {d} {start} {end} {output_file} p_step={step} mode={mode} dump=false"
    subprocess.Popen(["xterm", "-e", f'cd "{cwd}" && {command}; bash'])


/home/tommaso-peduzzi/Documents/clayg/data/treshold_plots/11-2025-06-23_13-42-07
/home/tommaso-peduzzi/Documents/clayg/data/treshold_plots/11-2025-06-23_13-42-07/d_4.txt
/home/tommaso-peduzzi/Documents/clayg/data/treshold_plots/11-2025-06-23_13-42-07/d_8.txt
/home/tommaso-peduzzi/Documents/clayg/data/treshold_plots/11-2025-06-23_13-42-07/d_10.txt
/home/tommaso-peduzzi/Documents/clayg/data/treshold_plots/11-2025-06-23_13-42-07/d_12.txt


In [3]:
import glob
import re
import os
import sys
import plotly.graph_objects as go

decoder_colors = {
    'uf': [
        "#deebf7", "#c6dbef", "#9ecae1", "#6baed6", "#4292c6",
        "#3182bd", "#1f77b4", "#2171b5", "#08519c", "#08306b",
        "#ffcccc", "#ff9999", "#ff6666", "#ff3333", "#ff0000"
    ],
    'clayg': [
        "#fdae6b", "#ffbb78", "#ff8c00", "#fd8d3c", "#ffa726",
        "#f16913", "#ff7f0e", "#d95f02", "#d94801", "#a63603",
        "#ccffcc", "#99ff99", "#66ff66", "#33ff33", "#00ff00"
    ]
}

# rainbow
decoder_colors["clayg"] = [
    "#ff0000", "#ff7f00", "#0e550e", "#0000ff",
    "#4b0082", "#9400d3", "#8b00ff", "#ff00ff", "#ff1493",
    "#ff69b4", "#ffc0cb", "#ffa07a", "#fa8072", "#f08080"
]

smoothing_factor = 0.5

In [19]:
plot_ids = [11]

fig = go.Figure()

title = f"Threshold Plot ClAYG and Union Find\n"
title += f"(10000 trials per point)"

i = 0
for plot_id in plot_ids:
    data_folder = os.path.join("data/treshold_plots", f"{plot_id}-*")
    matching_folders = glob.glob(data_folder)
    if not matching_folders:
        print(f"No folder found with ID {plot_id}")
        continue

    folder_name = os.path.basename(matching_folders[0])
    data_by_depth = {}
    files = glob.glob(os.path.join(matching_folders[0], "d=*.txt"))
    files += glob.glob(os.path.join(matching_folders[0], "d_*.txt"))
    pattern = re.compile(r"d[=_](\d+)\.txt")

    # each file is in the format 
    # d=18.txt or d_18.txt
    # and contains the data for a specific depth in the format
    # p decoder1 decoder2 decdoder3
    # we want to extract the depth from the filename
    # and the data from the file
    for file in files:
        match = pattern.search(file)
        if not match:
            continue
        depth = int(match.group(1))
        with open(file, "r") as f:
            # read the first line to get the header
            header = f.readline().strip().split()
            # extract the decoders from the first line
            decoders = header[1:]
            data = {}
            for line in f:
                # read the data from the file
                text = line.strip().split()
                # extract the p value and the decoder values
                p = float(text[0])
                values = list(map(float, text[1:]))
                # data by depth strucuture [depth][p] = dict[decoder1, decoder2, decoder3]
                if p not in data:
                    data[p] = {}
                for value, decoder in zip(values, decoders[:len(values)]):
                    data[p][decoder] = value
        data_by_decoder = {}
        for p, decoders in data.items():
            for decoder, value in decoders.items():
                if not decoder in data_by_decoder:
                    data_by_decoder[decoder] = {}
                data_by_decoder[decoder][p] = value
        
        names = {
            "uf": "Union Find",
            "clayg": "ClAYG"
        }
        print (f"Plotting data for depth {depth} in folder {folder_name}")
        
        for decoder, values in data_by_decoder.items():
            if not decoder in ["uf", "clayg"]:
                continue
            name = names[decoder]
            i += 1
            fig.add_trace(go.Scatter(
                x=list(values.keys()), y=list(values.values()), mode="lines+markers",
                name=f"{name} d={depth}",
                line=dict(color=decoder_colors[decoder][i % len(decoder_colors[decoder])], shape="spline", smoothing=smoothing_factor, width=4),
                legendgroup=f"{name} {plot_id}",
                legendgrouptitle_text=f"{name}",
            ))
            
            def curve(log_p, C, P):
                return C*log_p + P
            
            import numpy as np
            log_p = np.log(list(values.keys()))
            log_pl = np.log(list(values.values()))
            
            # remove all inf and nans
            log_p, log_pl = log_p[np.isfinite(log_p) & np.isfinite(log_pl)], log_pl[np.isfinite(log_p) & np.isfinite(log_pl)]
            
            # Fit the curve
            from scipy.optimize import curve_fit
            try:
                _, C = np.polyfit(log_p, log_pl, 1)
                # Generate points for the fitted curve
                print(f"C = {C}, (d+1)/2 = {(depth+1)/2}, minimum p = {np.exp(log_p.min())}")
            except:
                print(f"Could not fit curve for {name} d={depth}")
                continue

# add a vertical line at p=0.017
fig.add_shape(
    type="line", x0=0.016, x1=0.016, y0=0, y1=0.4,
    line=dict(color=decoder_colors['clayg'][0], width=4, dash="dash"),
    legendgroup="Thresholds",
    name="Threshold ClAYG"
)
fig.add_shape(
    type="line", x0=0.0242, x1=0.0242, y0=0, y1=0.4,
    line=dict(color=decoder_colors['uf'][2], width=4    , dash="dash"),
    name="Threshold Union Find",
    legendgroup="Thresholds"
)

fig.update_layout(
    title=title,
    xaxis_title="Physical Error Rate (p)",
    yaxis_title="Logical Error Rate (l)",
    template="plotly_white",
    hoverlabel_namelength=-1,
    legend=dict(groupclick="toggleitem"),
    font=dict(size=20),
)

loglog = False
if loglog:
    fig.update_layout(xaxis_type="log", yaxis_type="log")
else:
    fig.update_layout(
        yaxis=dict(range=[0, 0.18], type="linear"),
        xaxis=dict(range=[0, 0.035], type="linear"),
    ) 

fig.show(renderer="browser")


Plotting data for depth 10 in folder 11-2025-06-23_13-42-07
Could not fit curve for ClAYG d=10
Plotting data for depth 12 in folder 11-2025-06-23_13-42-07
Could not fit curve for ClAYG d=12
Plotting data for depth 4 in folder 11-2025-06-23_13-42-07
C = 4.6227601044913165, (d+1)/2 = 2.5, minimum p = 0.00035000000000000005
Plotting data for depth 8 in folder 11-2025-06-23_13-42-07
Could not fit curve for ClAYG d=8


/tmp/ipykernel_5563/2035744675.py:81: RuntimeWarning:

divide by zero encountered in log



Opening in existing browser session.


In [ ]:
import numpy as np


plot_ids = [11]

title = "Under Threshold Plot ClAYG\n"
title += f"(10000 trials per point)"

for plot_id in plot_ids:
    data_folder = os.path.join("data/treshold_plots", f"{plot_id}-*")
    matching_folders = glob.glob(data_folder)
    if not matching_folders:
        print(f"No folder found with ID {plot_id}")
        continue

    folder_name = os.path.basename(matching_folders[0])
    data_by_probability = {}  # [decoder][p][depth] = value
    files = glob.glob(os.path.join(matching_folders[0], "d=*.txt"))
    files += glob.glob(os.path.join(matching_folders[0], "d_*.txt"))
    pattern = re.compile(r"d[=_](\d+)\.txt")

    # each file is in the format
    # d=18.txt or d_18.txt
    # and contains the data for a specific depth in the format
    # p decoder1 decoder2 decdoder3
    # we want to extract the depth from the filename
    # and the data from the file
    for file in files:
        match = pattern.search(file)
        if not match:
            continue
        depth = int(match.group(1))
        with open(file, "r") as f:
            # read the first line to get the header
            header = f.readline().strip().split()
            # extract the decoders from the first line
            decoders = header[1:]
            for line in f:
                data = {}
                # read the data from the file
                text = line.strip().split()
                # extract the p value and the decoder values
                p = float(text[0])
                values = list(map(float, text[1:]))
                # data by depth strucuture [depth][p] = dict[decoder1, decoder2, decoder3]
                for i in range(len(values)):
                    decoder = decoders[i]
                    data[decoder] = values[i]
                for decoder in decoders:
                    if not decoder in data_by_probability:
                        data_by_probability[decoder] = {}
                    if not p in data_by_probability[decoder]:
                        data_by_probability[decoder][p] = {}
                    if decoder in data:
                        data_by_probability[decoder][p][depth] = data[decoder]

fig = go.Figure()
title = f"Under Threshold Plot ClAYG\n"
title += f"(10000 trials per point)"

data = data_by_probability['clayg']
probabilities = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06]
i = 0
for p, depths in data.items():
    if not (p > 0.005 and p <= 0.016):
        continue
    color_index = i % len(decoder_colors[decoder])
    print(depths)
    sorted_depths_by_depths = dict(sorted(depths.items()))
    
    # regression over depths
    depths_array = np.array(list(sorted_depths_by_depths.keys()))
    values_array = np.array(list(sorted_depths_by_depths.values()))
    if len(depths_array) < 2:
        continue
    # fit exponential regression
    coefficients = np.polyfit(np.log(depths_array), np.log(values_array), 1)
    regression_line = np.exp(coefficients[1]) * (depths_array ** coefficients[0])
    # draw regression line
    fig.add_trace(go.Scatter(
        x=depths_array, y=regression_line, mode="lines",
        name=f"Regression p={p}",
        marker=dict(color=decoder_colors['clayg'][color_index], size=10),
        line=dict(color=decoder_colors[decoder][color_index], shape="linear", smoothing=smoothing_factor, width=3),
    ))
    print(f"p={p}, depths={sorted_depths_by_depths}")
    
    fig.add_trace(go.Scatter(
        x=list(sorted_depths_by_depths.keys()), y=list(sorted_depths_by_depths.values()), mode="lines+markers",
        name=f"p={p}",
        marker=dict(color=decoder_colors['clayg'][color_index], size=10),
    ))
    i += 1
    
fig.update_layout(xaxis_type="log", yaxis_type="log")
fig.update_layout(
    title=title,
    xaxis_title="Code Size (d)",
    yaxis_title="Logical Error Rate (l)",
    template="plotly_white",
    hoverlabel_namelength=-1,
    legend=dict(groupclick="toggleitem"),
    font=dict(size=20),
)
fig.show(renderer="browser")

NameError: name 'os' is not defined